In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import FancyURLopener

# change fighter url 
# url = "https://www.sherdog.com/events/UFC-Fight-Night-101-Whittaker-vs-Brunson-54995"
url = "https://www.sherdog.com/fighter/Jon-Jones-27944"
fighter = url[32:-6]

# html = urlopen(url)
class MyOpener(FancyURLopener):
    version = 'My new User-Agent'

# spoof as browser to access url     
MyOpener.version
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'    
    
# open url and parse with bs4     
myopener = MyOpener()
page = myopener.open(url)
soup = BeautifulSoup(page, 'lxml')    
text = soup.get_text()

# find all table rows and assign to rows
rows = soup.find_all('tr')

str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    
    
import re

# further parse with regex
list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
    
# turn parsed text into df    
df = pd.DataFrame(list_rows)    

# delimit text and remove square brackets
df1 = df[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[')
df1[5] = df1[5].str.strip(']')
df1.columns = df1.iloc[2]
# df1 = df1.reindex(df1.index.drop(2))
# df1.head(10)

# trim excess rows at the ends not pertaining to fight record
# will need to adjust dropping 0,1,2 for fighters who do not have an upcoming fight 
df2 = df1.drop([0,1,2,df.shape[0]-3,df.shape[0]-2,df.shape[0]-1],axis = 0)

cols=list(df2.columns)

# split out method and referee into 3 columns

df2[cols[3]] = df2[cols[3]].str.replace(')', '))')
methref = df2[cols[3]].str.rsplit(')', expand=True, n=1)
methref.columns = ['method','Referee']
fin = methref['method'].str.split('(',expand = True)
fin[1] = fin[1].str.strip(')')
fin.columns = ['Method','Finish']

# split up event and date

ev = df2[cols[2]].str.rsplit('/', expand=True)
ev['mth'] = ev[0].str[-4:]
ev[0] = ev[0].str[:-4]

#rearranging column order
ev.columns = ['Event','Day','Year','Month']
ev = ev[['Event','Month','Day','Year']]

#join all 4 tables together dropping excess event and method/referee columns that have been cleaned 
newtbl = pd.concat([df2.drop([cols[2], cols[3]],axis = 1) ,ev,fin,methref['Referee']],axis = 1)


# create fighter-name.csv
csvname = '.\\{}.csv'.format(fighter)
newtbl.to_csv(path_or_buf = csvname,index = False)



C:\Users\Vincent.Cheng\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


In [57]:


for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td height="37" width="37"><img data-src="{image}" width="32"/></td>, <td>{name} - {title}</td>]


bs4.element.ResultSet

In [59]:

print(list_rows)
type(clean2)

['[Date, VS. Fighter, Event, Location]', '[\n\n\n\nUFC 239\n\n\n\n\nJuly 06, 2019 / T-Mobile Arena, Las Vegas, United States, United States\n\n\n\nVersus\n\n\nJon Jones\n24 - 1 - 0\n(Win - Loss - Draw)\n\n\n\nThiago Santos\n21 - 6 - 0\n(Win - Loss - Draw)\n\n\n\nSee entire fight card\n\n\n\n\n, \n]', '[Result, Fighter, Event, Method/Referee, R, Time]', '[win, Anthony Smith, UFC 235 - Jones vs. SmithMar / 02 / 2019, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Alexander Gustafsson, UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018, KO (Punches)Mike Beltran, 3, 2:02]', '[NC, Daniel Cormier, UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017, No Contest (Overturned by CSAC)John McCarthy, 3, 3:01]', '[win, Ovince St. Preux, UFC 197 - Jones vs. St. PreuxApr / 23 / 2016, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Daniel Cormier, UFC 182 - Jones vs. CormierJan / 03 / 2015, Decision (Unanimous)Herb Dean, 5, 5:00]', '[win, Glover Teixeira, UFC 172 - Jones vs. TeixeiraApr / 26 / 2014, Decision

str

In [60]:

# df.columns = df.iloc[0]
# df.reindex(df.index.drop(0))

df.head(10)

,0
0,"[Date, VS. Fighter, Event, Location]"
1,"[\n\n\n\nUFC 239\n\n\n\n\nJuly 06, 2019 / T-Mo..."
2,"[Result, Fighter, Event, Method/Referee, R, Time]"
3,"[win, Anthony Smith, UFC 235 - Jones vs. Smith..."
4,"[win, Alexander Gustafsson, UFC 232 - Jones vs..."
5,"[NC, Daniel Cormier, UFC 214 - Cormier vs. Jon..."
6,"[win, Ovince St. Preux, UFC 197 - Jones vs. St..."
7,"[win, Daniel Cormier, UFC 182 - Jones vs. Corm..."
8,"[win, Glover Teixeira, UFC 172 - Jones vs. Tei..."
9,"[win, Alexander Gustafsson, UFC 165 - Jones vs..."


In [62]:
df1

2,Result,Fighter,Event,Method/Referee,R,Time
0,Date,VS. Fighter,Event,Location],None,None
1,\n\n\n\nUFC 239\n\n\n\n\nJuly 06,2019 / T-Mobile Arena,Las Vegas,United States,United States\n\n\n\nVersus\n\n\nJon Jones\n2...,\n
2,Result,Fighter,Event,Method/Referee,R,Time
3,win,Anthony Smith,UFC 235 - Jones vs. SmithMar / 02 / 2019,Decision (Unanimous)Herb Dean,5,5:00
4,win,Alexander Gustafsson,UFC 232 - Jones vs. Gustafsson 2Dec / 29 / 2018,KO (Punches)Mike Beltran,3,2:02
5,NC,Daniel Cormier,UFC 214 - Cormier vs. Jones 2Jul / 29 / 2017,No Contest (Overturned by CSAC)John McCarthy,3,3:01
6,win,Ovince St. Preux,UFC 197 - Jones vs. St. PreuxApr / 23 / 2016,Decision (Unanimous)Herb Dean,5,5:00
7,win,Daniel Cormier,UFC 182 - Jones vs. CormierJan / 03 / 2015,Decision (Unanimous)Herb Dean,5,5:00
8,win,Glover Teixeira,UFC 172 - Jones vs. TeixeiraApr / 26 / 2014,Decision (Unanimous)Dan Miragliotta,5,5:00
9,win,Alexander Gustafsson,UFC 165 - Jones vs. GustafssonSep / 21 / 2013,Decision (Unanimous)John McCarthy,5,5:00


In [65]:

newtbl

,Result,Fighter,R,Time,Event,Month,Day,Year,Method,Finish,Referee
3,win,Anthony Smith,5,5:00,UFC 235 - Jones vs. Smith,Mar,02,2019,Decision,Unanimous,Herb Dean
4,win,Alexander Gustafsson,3,2:02,UFC 232 - Jones vs. Gustafsson 2,Dec,29,2018,KO,Punches,Mike Beltran
5,NC,Daniel Cormier,3,3:01,UFC 214 - Cormier vs. Jones 2,Jul,29,2017,No Contest,Overturned by CSAC,John McCarthy
6,win,Ovince St. Preux,5,5:00,UFC 197 - Jones vs. St. Preux,Apr,23,2016,Decision,Unanimous,Herb Dean
7,win,Daniel Cormier,5,5:00,UFC 182 - Jones vs. Cormier,Jan,03,2015,Decision,Unanimous,Herb Dean
8,win,Glover Teixeira,5,5:00,UFC 172 - Jones vs. Teixeira,Apr,26,2014,Decision,Unanimous,Dan Miragliotta
9,win,Alexander Gustafsson,5,5:00,UFC 165 - Jones vs. Gustafsson,Sep,21,2013,Decision,Unanimous,John McCarthy
10,win,Chael Sonnen,1,4:33,UFC 159 - Jones vs. Sonnen,Apr,27,2013,TKO,Elbows and Punches,Keith Peterson
11,win,Vitor Belfort,4,0:54,UFC 152 - Jones vs. Belfort,Sep,22,2012,Submission,Keylock,John McCarthy
12,win,Rashad Evans,5,5:00,UFC 145 - Jones vs. Evans,Apr,21,2012,Decision,Unanimous,Herb Dean
